In [1]:
# 1. EDA: Sanity check splits and class balance
# =====================================================
# CHOOSE YOUR DATASET:
# - FILTERED: High quality labels, smaller size (25K), 86% hate_type coverage
# - ENHANCED: Auto-labeled, larger size (75K), 95% hate_type coverage ⭐ RECOMMENDED
# =====================================================
import pandas as pd

# OPTION 1: Filtered dataset (no toxic_comments) - Fast training, high quality
# df = pd.read_csv('dataset/UNIFIED_ALL_SPLIT_FILTERED.csv')

# OPTION 2: Enhanced dataset (with auto-labeled toxic_comments) - Best performance ⭐
df = pd.read_csv('dataset/UNIFIED_ALL_SPLIT_ENHANCED.csv')

print(f'📊 Dataset: {df["source_dataset"].nunique()} sources, {len(df)} total samples')
print('='*60)
print(f'Total samples: {len(df)}\n')

print('Split counts:')
print(df['split'].value_counts(), '\n')

print('Per split language distribution:')
print(df.groupby('split')['language'].value_counts(), '\n')

print('is_hate by split:')
print(df.groupby('split')['is_hate'].value_counts(), '\n')

print('hate_type distribution (valid labels):')
ht_valid = df[df['hate_type'] != -1]
print(f'{len(ht_valid)}/{len(df)} samples have hate_type labels ({len(ht_valid)/len(df)*100:.1f}%)')
print(ht_valid['hate_type'].value_counts().sort_index(), '\n')

print('target_group distribution (valid labels):')
tg_valid = df[df['target_group'] != -1]
print(f'{len(tg_valid)}/{len(df)} samples have target_group labels ({len(tg_valid)/len(df)*100:.1f}%)')
print(tg_valid['target_group'].value_counts().sort_index(), '\n')

print('severity distribution:')
print(df['severity'].value_counts().sort_index(), '\n')

print('source_dataset distribution:')
print(df['source_dataset'].value_counts())

print('\n' + '='*60)
if 'toxic_comments_labeled' in df['source_dataset'].values:
    print('✅ Using ENHANCED dataset with auto-labeled toxic_comments')
    print('   - 95% hate_type coverage')
    print('   - 77% target_group coverage')
    print('   - Best for final model training!')
else:
    print('✅ Using FILTERED dataset (toxic_comments excluded)')
    print('   - 86% hate_type coverage')
    print('   - High label quality')
    print('   - Fast training!')
print('='*60)


📊 Dataset: 6 sources, 75864 total samples
Total samples: 75864

Split counts:
split
train    45518
val      15173
test     15173
Name: count, dtype: int64 

Per split language distribution:
split  language
test   english      6158
       bangla       5679
       banglish     3336
train  english     18475
       bangla      17036
       banglish    10007
val    english      6159
       bangla       5678
       banglish     3336
Name: count, dtype: int64 

is_hate by split:
split  is_hate
test   0          10139
       1           5034
train  0          30418
       1          15100
val    0          10141
       1           5032
Name: count, dtype: int64 

hate_type distribution (valid labels):
72354/75864 samples have hate_type labels (95.4%)
hate_type
0    51271
1     1382
2     1409
3      873
4    14357
5     3062
Name: count, dtype: int64 

target_group distribution (valid labels):
58669/75864 samples have target_group labels (77.3%)
target_group
0    42826
1    12523
2     2183
3 

In [2]:
# COLAB SETUP: Mount Google Drive and set paths
# =====================================================
# Run this cell FIRST on Colab!
# =====================================================

from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory on Google Drive
import os
COLAB_CHECKPOINT_DIR = '/content/drive/MyDrive/thesis_training/checkpoints_v2/'
os.makedirs(COLAB_CHECKPOINT_DIR, exist_ok=True)

# Set this to resume from a previous epoch (or None to start fresh)
resume_checkpoint = None  # e.g., '/content/drive/MyDrive/thesis_training/checkpoints_v2/xlmr_v2_epoch2.pt'

print(f'✅ Google Drive mounted!')
print(f'📁 Checkpoints will be saved to: {COLAB_CHECKPOINT_DIR}')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# 2. HateDataset: PyTorch Dataset with tokenization and masking for incomplete labels
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer

class HateDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=160):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row['text'])
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Extract labels (use -1 for missing)
        hate_type = int(row['hate_type'])
        target_group = int(row['target_group'])
        severity = int(row['severity'])
        
        # Create masks: True if label is valid (not -1)
        hate_type_mask = hate_type != -1
        target_group_mask = target_group != -1
        severity_mask = severity != -1
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'hate_type': torch.tensor(max(0, hate_type), dtype=torch.long),
            'target_group': torch.tensor(max(0, target_group), dtype=torch.long),
            'severity': torch.tensor(max(0, severity), dtype=torch.long),
            'hate_type_mask': torch.tensor(hate_type_mask, dtype=torch.bool),
            'target_group_mask': torch.tensor(target_group_mask, dtype=torch.bool),
            'severity_mask': torch.tensor(severity_mask, dtype=torch.bool),
        }

# Test the dataset
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
test_df = df.head(5)
test_dataset = HateDataset(test_df, tokenizer)
sample = test_dataset[0]
print('Sample keys:', sample.keys())
print('input_ids shape:', sample['input_ids'].shape)
print('hate_type:', sample['hate_type'].item(), 'mask:', sample['hate_type_mask'].item())

Sample keys: dict_keys(['input_ids', 'attention_mask', 'hate_type', 'target_group', 'severity', 'hate_type_mask', 'target_group_mask', 'severity_mask'])
input_ids shape: torch.Size([160])
hate_type: 4 mask: True


In [4]:
# 3. MultiTaskXLMRRoberta: Model with shared backbone and task-specific heads
import torch.nn as nn
from transformers import XLMRobertaModel

class MultiTaskXLMRRoberta(nn.Module):
    def __init__(self, model_name='xlm-roberta-large', dropout=0.2,
                 n_hate_type=6, n_target_group=4, n_severity=4):
        super().__init__()
        self.backbone = XLMRobertaModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size  # 1024 for large
        
        self.dropout = nn.Dropout(dropout)
        
        # Task-specific classification heads
        self.hate_type_head = nn.Linear(hidden_size, n_hate_type)
        self.target_group_head = nn.Linear(hidden_size, n_target_group)
        self.severity_head = nn.Linear(hidden_size, n_severity)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        # Use CLS token representation
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        
        hate_type_logits = self.hate_type_head(cls_output)
        target_group_logits = self.target_group_head(cls_output)
        severity_logits = self.severity_head(cls_output)
        
        return hate_type_logits, target_group_logits, severity_logits

# Instantiate and check model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model = MultiTaskXLMRRoberta().to(device)
print(f'Model loaded. Backbone hidden size: {model.backbone.config.hidden_size}')
print(model)

Using device: cpu
Model loaded. Backbone hidden size: 1024
MultiTaskXLMRRoberta(
  (backbone): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
     

In [5]:
# 4. multitask_loss: Masked cross-entropy loss with CLASS WEIGHTS for imbalanced data
import torch.nn.functional as F

def compute_class_weights(df, column, n_classes, smoothing=0.1):
    """Compute inverse frequency class weights with smoothing."""
    valid = df[df[column] != -1][column]
    counts = valid.value_counts().reindex(range(n_classes), fill_value=1).values
    weights = 1.0 / (counts + smoothing * len(valid))
    weights = weights / weights.sum() * n_classes  # Normalize to sum to n_classes
    return torch.tensor(weights, dtype=torch.float32)

def multitask_loss(hate_type_logits, target_group_logits, severity_logits,
                   targets, masks, task_weights=(1.0, 1.0, 1.0),
                   ht_class_weights=None, tg_class_weights=None, sv_class_weights=None):
    """
    Compute masked cross-entropy loss with optional class weights.
    Class weights help the model pay more attention to minority classes.
    """
    total_loss = 0.0
    n_tasks = 0
    
    # Hate type loss (only where mask is True)
    ht_mask = masks['hate_type'].bool()
    if ht_mask.any():
        loss_ht = F.cross_entropy(
            hate_type_logits[ht_mask], 
            targets['hate_type'][ht_mask],
            weight=ht_class_weights
        )
        total_loss += task_weights[0] * loss_ht
        n_tasks += 1
    
    # Target group loss
    tg_mask = masks['target_group'].bool()
    if tg_mask.any():
        loss_tg = F.cross_entropy(
            target_group_logits[tg_mask], 
            targets['target_group'][tg_mask],
            weight=tg_class_weights
        )
        total_loss += task_weights[1] * loss_tg
        n_tasks += 1
    
    # Severity loss
    sv_mask = masks['severity'].bool()
    if sv_mask.any():
        loss_sv = F.cross_entropy(
            severity_logits[sv_mask], 
            targets['severity'][sv_mask],
            weight=sv_class_weights
        )
        total_loss += task_weights[2] * loss_sv
        n_tasks += 1
    
    return total_loss / max(1, n_tasks)

print('✅ multitask_loss function defined with CLASS WEIGHTS support.')

✅ multitask_loss function defined with CLASS WEIGHTS support.


In [6]:
# 5. Mini-batch validation: Test forward pass and loss computation
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)
batch = next(iter(test_loader))

def move_batch_to_device(batch):
    return {k: v.to(device) for k, v in batch.items()}

batch = move_batch_to_device(batch)

model.eval()
with torch.no_grad():
    ht_logits, tg_logits, sv_logits = model(batch['input_ids'], batch['attention_mask'])

print('Logit shapes:')
print(f'  hate_type: {ht_logits.shape}')
print(f'  target_group: {tg_logits.shape}')
print(f'  severity: {sv_logits.shape}')

targets = {k: batch[k] for k in ['hate_type', 'target_group', 'severity']}
masks = {k: batch[f'{k}_mask'] for k in targets.keys()}
loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks)
print(f'Batch loss: {loss.item():.4f}')

Logit shapes:
  hate_type: torch.Size([2, 6])
  target_group: torch.Size([2, 4])
  severity: torch.Size([2, 4])
Batch loss: 1.3877


In [7]:
# 6. Full data loaders setup + COMPUTE CLASS WEIGHTS
import os
SEED = 1337
MAX_LENGTH = 160
BATCH_SIZE = 16

# Use Colab checkpoint dir if available, otherwise local
try:
    CHECKPOINT_DIR = COLAB_CHECKPOINT_DIR
    print(f'Using Colab checkpoint dir: {CHECKPOINT_DIR}')
except NameError:
    CHECKPOINT_DIR = 'checkpoints/'
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    print(f'Using local checkpoint dir: {CHECKPOINT_DIR}')

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

train_df = df[df['split'] == 'train'].reset_index(drop=True)
val_df = df[df['split'] == 'val'].reset_index(drop=True)
test_df = df[df['split'] == 'test'].reset_index(drop=True)
print(f'Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

# Compute class weights from training data (helps with imbalanced classes!)
ht_weights = compute_class_weights(train_df, 'hate_type', 6).to(device)
tg_weights = compute_class_weights(train_df, 'target_group', 4).to(device)
sv_weights = compute_class_weights(train_df, 'severity', 4).to(device)

print(f'\n📊 Class Weights (higher = more focus on that class):')
print(f'  hate_type:    {[f"{w:.2f}" for w in ht_weights.tolist()]}')
print(f'  target_group: {[f"{w:.2f}" for w in tg_weights.tolist()]}')
print(f'  severity:     {[f"{w:.2f}" for w in sv_weights.tolist()]}')

train_dataset = HateDataset(train_df, tokenizer, max_length=MAX_LENGTH)
val_dataset = HateDataset(val_df, tokenizer, max_length=MAX_LENGTH)
test_dataset = HateDataset(test_df, tokenizer, max_length=MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
print(f'\nData loaders created. Batches - Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}')

Using local checkpoint dir: checkpoints/
Train: 45518, Val: 15173, Test: 15173

📊 Class Weights (higher = more focus on that class):
  hate_type:    ['0.20', '1.35', '1.34', '1.44', '0.54', '1.14']
  target_group: ['0.24', '0.64', '1.45', '1.67']
  severity:     ['0.23', '0.78', '1.62', '1.37']

Data loaders created. Batches - Train: 2845, Val: 949, Test: 949


In [8]:
# 7. evaluate() helper: Compute loss and F1 metrics (with class weights)
from sklearn.metrics import f1_score, classification_report
import numpy as np

def evaluate(model, data_loader, task_weights=(1.0, 1.0, 1.0),
             ht_class_weights=None, tg_class_weights=None, sv_class_weights=None,
             verbose=False):
    model.eval()
    total_loss = 0.0
    n_batches = 0
    all_preds = {'hate_type': [], 'target_group': [], 'severity': []}
    all_labels = {'hate_type': [], 'target_group': [], 'severity': []}
    all_masks = {'hate_type': [], 'target_group': [], 'severity': []}
    
    with torch.no_grad():
        for batch in data_loader:
            batch = move_batch_to_device(batch)
            ht_logits, tg_logits, sv_logits = model(batch['input_ids'], batch['attention_mask'])
            targets = {k: batch[k] for k in ['hate_type', 'target_group', 'severity']}
            masks = {k: batch[f'{k}_mask'] for k in targets.keys()}
            loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks, task_weights,
                                  ht_class_weights, tg_class_weights, sv_class_weights)
            total_loss += loss.item()
            n_batches += 1
            all_preds['hate_type'].extend(ht_logits.argmax(dim=1).cpu().numpy())
            all_preds['target_group'].extend(tg_logits.argmax(dim=1).cpu().numpy())
            all_preds['severity'].extend(sv_logits.argmax(dim=1).cpu().numpy())
            for task in ['hate_type', 'target_group', 'severity']:
                all_labels[task].extend(targets[task].cpu().numpy())
                all_masks[task].extend(masks[task].cpu().numpy())
    
    metrics = {'loss': total_loss / max(1, n_batches)}
    for task in ['hate_type', 'target_group', 'severity']:
        mask = np.array(all_masks[task]).astype(bool)
        if mask.sum() > 0:
            preds = np.array(all_preds[task])[mask]
            labels = np.array(all_labels[task])[mask]
            metrics[f'{task}_macro_f1'] = f1_score(labels, preds, average='macro', zero_division=0)
            metrics[f'{task}_micro_f1'] = f1_score(labels, preds, average='micro', zero_division=0)
            
            # Per-class F1 for detailed analysis
            if verbose:
                print(f'\n{task.upper()} Classification Report:')
                print(classification_report(labels, preds, zero_division=0))
        else:
            metrics[f'{task}_macro_f1'] = None
            metrics[f'{task}_micro_f1'] = None
    return metrics

print('✅ evaluate() function defined with class weights support.')

✅ evaluate() function defined with class weights support.


In [ ]:
# 8. train_model() function: With SPACE-SAVING checkpoint strategy
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import time

training_config = {
    'epochs': 5,           # Increased from 3
    'learning_rate': 1e-5, # Lower LR for stability (was 2e-5)
    'weight_decay': 1e-2,
    'warmup_ratio': 0.1,
    'grad_clip': 1.0,
    'patience': 3,         # More patience
    'dropout': 0.3,        # Slightly higher dropout
    'task_weights': (1.0, 1.0, 1.0),
    'use_class_weights': True  # NEW: Enable class weights
}

def train_model(train_loader, val_loader, config=None, run_name='xlmr_run', use_wandb=False, resume_from=None,
                ht_class_weights=None, tg_class_weights=None, sv_class_weights=None):
    """
    Train the multi-task model with class weights for imbalanced data.
    SPACE-SAVING: Auto-deletes old epoch checkpoints to save disk space.
    """
    if config is None: config = training_config
    if use_wandb:
        import wandb
        wandb.init(project='multilingual-hate-detection', name=run_name, config=config, resume='allow')
    
    model = MultiTaskXLMRRoberta(dropout=config['dropout']).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    total_steps = len(train_loader) * config['epochs']
    warmup_steps = int(total_steps * config['warmup_ratio'])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    
    best_val_loss = float('inf')
    best_macro_f1 = 0.0  # Also track best macro F1
    patience_counter = 0
    start_epoch = 1
    history = []
    
    best_ckpt_path = os.path.join(CHECKPOINT_DIR, f'{run_name}_best.pt')
    
    # Resume from checkpoint if provided
    if resume_from and os.path.exists(resume_from):
        print(f'Resuming from checkpoint: {resume_from}')
        checkpoint = torch.load(resume_from, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_loss = checkpoint.get('best_val_loss', float('inf'))
        best_macro_f1 = checkpoint.get('best_macro_f1', 0.0)
        patience_counter = checkpoint.get('patience_counter', 0)
        history = checkpoint.get('history', [])
        print(f'Resumed from epoch {checkpoint["epoch"]}. Starting epoch {start_epoch}.')
    
    for epoch in range(start_epoch, config['epochs'] + 1):
        model.train()
        start = time.time()
        running_loss = 0.0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch}/{config["epochs"]}', leave=True)
        for batch_idx, batch in enumerate(pbar):
            batch = move_batch_to_device(batch)
            optimizer.zero_grad()
            logits = model(batch['input_ids'], batch['attention_mask'])
            targets = {k: batch[k] for k in ['hate_type', 'target_group', 'severity']}
            masks = {k: batch[f'{k}_mask'] for k in targets.keys()}
            
            # Use class weights if provided
            loss = multitask_loss(*logits, targets, masks, config['task_weights'],
                                  ht_class_weights, tg_class_weights, sv_class_weights)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
            optimizer.step()
            scheduler.step()
            running_loss += loss.item()
            
            avg_loss = running_loss / (batch_idx + 1)
            pbar.set_postfix({'loss': f'{avg_loss:.4f}'})
        
        train_loss = running_loss / max(1, len(train_loader))
        
        print(f'Evaluating on validation set...')
        val_metrics = evaluate(model, val_loader, config['task_weights'],
                               ht_class_weights, tg_class_weights, sv_class_weights)
        val_loss = val_metrics['loss']
        
        # Compute average macro F1 across tasks
        macro_f1s = [val_metrics.get(f'{t}_macro_f1', 0) or 0 for t in ['hate_type', 'target_group', 'severity']]
        avg_macro_f1 = sum(macro_f1s) / len(macro_f1s)
        
        epoch_time = time.time() - start
        log_payload = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss, 
                       'avg_macro_f1': avg_macro_f1, 'epoch_time': epoch_time, **val_metrics}
        history.append(log_payload)
        if use_wandb: wandb.log(log_payload)
        
        print(f'Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, avg_macro_f1={avg_macro_f1:.4f}, time={epoch_time:.1f}s')
        print(f'  hate_type_macro_f1={val_metrics.get("hate_type_macro_f1", 0):.4f}, target_group_macro_f1={val_metrics.get("target_group_macro_f1", 0):.4f}, severity_macro_f1={val_metrics.get("severity_macro_f1", 0):.4f}')
        
        # ⚡ SPACE-SAVING: Save epoch checkpoint (for resume)
        epoch_ckpt_path = os.path.join(CHECKPOINT_DIR, f'{run_name}_epoch{epoch}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_loss': best_val_loss,
            'best_macro_f1': best_macro_f1,
            'patience_counter': patience_counter,
            'history': history,
            'config': config
        }, epoch_ckpt_path)
        print(f'  💾 Epoch checkpoint saved to {epoch_ckpt_path}')
        
        # Save best model based on MACRO F1 (better for imbalanced data)
        if avg_macro_f1 > best_macro_f1:
            best_macro_f1 = avg_macro_f1
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_ckpt_path)
            print(f'  ✓ New best checkpoint saved! (avg_macro_f1={avg_macro_f1:.4f})')
            
            # ⚡ DELETE OLD EPOCH CHECKPOINT after saving best (saves 7GB per epoch!)
            if epoch > 1:
                old_epoch_ckpt = os.path.join(CHECKPOINT_DIR, f'{run_name}_epoch{epoch-1}.pt')
                if os.path.exists(old_epoch_ckpt):
                    os.remove(old_epoch_ckpt)
                    print(f'  🗑️ Deleted old checkpoint: {old_epoch_ckpt}')
        else:
            patience_counter += 1
            print(f'  No improvement. Patience: {patience_counter}/{config["patience"]}')
            if patience_counter >= config['patience']:
                print('Early stopping triggered.')
                break
    
    # ⚡ FINAL CLEANUP: Delete last epoch checkpoint, keep only best
    final_epoch_ckpt = os.path.join(CHECKPOINT_DIR, f'{run_name}_epoch{epoch}.pt')
    if os.path.exists(final_epoch_ckpt):
        os.remove(final_epoch_ckpt)
        print(f'🗑️ Training complete. Deleted final epoch checkpoint. Only keeping: {best_ckpt_path}')
    
    if use_wandb: wandb.finish()
    return best_ckpt_path, history

print('✅ train_model() defined with SPACE-SAVING checkpoint strategy!')
print('💾 Saves: Best model (~2.5GB) + Latest epoch for resume (~7GB)')
print('🗑️ Auto-deletes old epoch checkpoints after each epoch')
print('📊 Total space needed: ~10GB max (vs 35GB for 5 epochs)')


✅ train_model() defined with CLASS WEIGHTS + macro F1 early stopping.


In [10]:
# 9. Smoke test loaders
SMOKE_TRAIN_SIZE = 512
SMOKE_VAL_SIZE = 256
SMOKE_TEST_SIZE = 256

smoke_train_df = train_df.sample(n=SMOKE_TRAIN_SIZE, random_state=SEED)
smoke_val_df = val_df.sample(n=SMOKE_VAL_SIZE, random_state=SEED)
smoke_test_df = test_df.sample(n=SMOKE_TEST_SIZE, random_state=SEED)

smoke_train_loader = DataLoader(HateDataset(smoke_train_df, tokenizer, max_length=MAX_LENGTH), batch_size=8, shuffle=True)
smoke_val_loader = DataLoader(HateDataset(smoke_val_df, tokenizer, max_length=MAX_LENGTH), batch_size=8, shuffle=False)
smoke_test_loader = DataLoader(HateDataset(smoke_test_df, tokenizer, max_length=MAX_LENGTH), batch_size=8, shuffle=False)
print(f'Smoke test sizes: Train={len(smoke_train_df)}, Val={len(smoke_val_df)}, Test={len(smoke_test_df)}')

Smoke test sizes: Train=512, Val=256, Test=256


In [11]:
# 10. Smoke test training
quick_config = training_config.copy()
quick_config.update({'epochs': 1, 'patience': 1})

best_checkpoint, history = train_model(smoke_train_loader, smoke_val_loader, config=quick_config, run_name='xlmr_smoke', use_wandb=False)
print('Training history:', history)

Epoch 1/1: 100%|██████████| 64/64 [10:57<00:00, 10.27s/it, loss=1.0116]



Evaluating on validation set...
Epoch 1: train_loss=1.0116, val_loss=0.7571, avg_macro_f1=0.1877, time=718.4s
  hate_type_macro_f1=0.1365, target_group_macro_f1=0.2086, severity_macro_f1=0.2181
Epoch 1: train_loss=1.0116, val_loss=0.7571, avg_macro_f1=0.1877, time=718.4s
  hate_type_macro_f1=0.1365, target_group_macro_f1=0.2086, severity_macro_f1=0.2181
  💾 Epoch checkpoint saved to checkpoints/xlmr_smoke_epoch1.pt
  💾 Epoch checkpoint saved to checkpoints/xlmr_smoke_epoch1.pt
  ✓ New best checkpoint saved! (avg_macro_f1=0.1877)
Training history: [{'epoch': 1, 'train_loss': 1.0116422502323985, 'val_loss': 0.7571360222063959, 'avg_macro_f1': 0.18771649739118393, 'epoch_time': 718.3748588562012, 'loss': 0.7571360222063959, 'hate_type_macro_f1': 0.1365079365079365, 'hate_type_micro_f1': 0.6935483870967742, 'target_group_macro_f1': 0.20857988165680474, 'target_group_micro_f1': 0.7157360406091371, 'severity_macro_f1': 0.21806167400881057, 'severity_micro_f1': 0.7734375}]
  ✓ New best checkp

In [12]:
# 11. Evaluate smoke checkpoint
best_model = MultiTaskXLMRRoberta().to(device)
best_model.load_state_dict(torch.load(best_checkpoint, map_location=device))
val_results = evaluate(best_model, smoke_val_loader)
test_results = evaluate(best_model, smoke_test_loader)
print('Validation metrics:', val_results)
print('Test metrics:', test_results)

Validation metrics: {'loss': 0.7571360222063959, 'hate_type_macro_f1': 0.1365079365079365, 'hate_type_micro_f1': 0.6935483870967742, 'target_group_macro_f1': 0.20857988165680474, 'target_group_micro_f1': 0.7157360406091371, 'severity_macro_f1': 0.21806167400881057, 'severity_micro_f1': 0.7734375}
Test metrics: {'loss': 0.7242736108601093, 'hate_type_macro_f1': 0.13762927605409706, 'hate_type_micro_f1': 0.7032520325203252, 'target_group_macro_f1': 0.21137026239067055, 'target_group_micro_f1': 0.7323232323232324, 'severity_macro_f1': 0.21991247264770242, 'severity_micro_f1': 0.78515625}


In [9]:
# 12. W&B Login
try:
    import wandb
except ModuleNotFoundError:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'wandb'])
    import wandb
wandb.login(key='61dd3d59137a5043373cd8ecc8f74c4d1c620ea6')
print('W&B login successful!')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Currently logged in as: c221025 (c221025-international-islamic-university-chittagong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: c221025 (c221025-international-islamic-university-chittagong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B login successful!


In [ ]:
# 13. Full training WITH CLASS WEIGHTS (Enhanced Dataset)
# =====================================================
# Using ENHANCED dataset with auto-labeled toxic_comments
# Expected: 85% test F1 (hate_type), 74% (target_group), 95% (severity)
# =====================================================

full_training_config = {
    'epochs': 5,
    'learning_rate': 1e-5,      # Lower LR for stability with auto-labeled data
    'weight_decay': 1e-2,
    'warmup_ratio': 0.1,
    'grad_clip': 1.0,
    'patience': 3,              # Early stopping patience
    'dropout': 0.3,
    'task_weights': (1.0, 1.0, 1.0),
    'use_class_weights': True   # CRITICAL for handling class imbalance!
}

print(f'🚀 Starting ENHANCED training on {len(train_dataset)} samples...')
print(f'Device: {device}')
print(f'📁 Saving checkpoints to: {CHECKPOINT_DIR}')
print(f'\n📊 Using class weights to handle imbalance!')
print(f'  hate_type weights:    {[f"{w:.2f}" for w in ht_weights.tolist()]}')
print(f'  target_group weights: {[f"{w:.2f}" for w in tg_weights.tolist()]}')
print(f'  severity weights:     {[f"{w:.2f}" for w in sv_weights.tolist()]}')

best_checkpoint_full, history_full = train_model(
    train_loader, val_loader, 
    config=full_training_config,
    run_name='xlmr_enhanced',      # ← Changed name to indicate enhanced dataset
    use_wandb=True,                # Set to False if no W&B
    ht_class_weights=ht_weights,
    tg_class_weights=tg_weights,
    sv_class_weights=sv_weights
)

print('\n✅ Training complete!')
print(f'📁 Best checkpoint saved to: {best_checkpoint_full}')
print('History:', history_full)


Starting full training on 45518 samples...
Device: cpu


Epoch 1/5: 100%|██████████| 2845/2845 [14:00:57<00:00, 17.74s/it, loss=0.6686]  



Evaluating on validation set...
Epoch 1: train_loss=0.6686, val_loss=0.4283, time=53392.9s
  hate_type_macro_f1=0.6151, target_group_macro_f1=0.5770, severity_macro_f1=0.6358
Epoch 1: train_loss=0.6686, val_loss=0.4283, time=53392.9s
  hate_type_macro_f1=0.6151, target_group_macro_f1=0.5770, severity_macro_f1=0.6358
  ✓ New best checkpoint saved to checkpoints/xlmr_full_large_best.pt
  ✓ New best checkpoint saved to checkpoints/xlmr_full_large_best.pt


Epoch 2/5:   1%|          | 35/2845 [11:10<14:56:47, 19.15s/it, loss=0.3921]



KeyboardInterrupt: 

In [ ]:
# 14. Load checkpoint and evaluate with DETAILED per-class metrics
# =====================================================
# For Colab: Use the checkpoint from training
# For Local: Download checkpoint from Google Drive first
# =====================================================

# Colab path (after training)
# CHECKPOINT_PATH = '/content/drive/MyDrive/thesis_training/checkpoints_v2/xlmr_v2_classweights_best.pt'

# Local path (after downloading from Google Drive)
CHECKPOINT_PATH = 'checkpoints/xlmr_v2_classweights_best.pt'

print(f'Loading checkpoint: {CHECKPOINT_PATH}')
best_model_full = MultiTaskXLMRRoberta().to(device)
best_model_full.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=device, weights_only=True))
print('✅ Model loaded successfully!')

print('\n' + '='*60)
print('=== VALIDATION SET RESULTS ===')
print('='*60)
val_results_full = evaluate(best_model_full, val_loader, verbose=True,
                            ht_class_weights=ht_weights, tg_class_weights=tg_weights, sv_class_weights=sv_weights)
print('\nSummary Metrics:')
for k, v in val_results_full.items():
    if v is not None and not k.startswith('_'): print(f'  {k}: {v:.4f}')

print('\n' + '='*60)
print('=== TEST SET RESULTS ===')
print('='*60)
test_results_full = evaluate(best_model_full, test_loader, verbose=True,
                             ht_class_weights=ht_weights, tg_class_weights=tg_weights, sv_class_weights=sv_weights)
print('\nSummary Metrics:')
for k, v in test_results_full.items():
    if v is not None and not k.startswith('_'): print(f'  {k}: {v:.4f}')

Loading checkpoint: checkpoints/xlmr_colab_best.pt
✅ Model loaded successfully!

=== Validation Set Results ===
✅ Model loaded successfully!

=== Validation Set Results ===


KeyboardInterrupt: 

In [35]:
# 15. 🔮 Inference: Predict on new text
# CORRECTED LABELS based on main.py mapping!
HATE_TYPE_LABELS = {
    0: 'not_hate/other',
    1: 'political',       # Political=1 in main.py
    2: 'religious',       # Religious=2 in main.py
    3: 'gender',          # Gender abusive=3 in main.py
    4: 'personal_attack', # Personal=4 in main.py  ← THIS IS WHAT MODEL PREDICTS!
    5: 'geopolitical'     # Geopolitical=5 in main.py
}
TARGET_GROUP_LABELS = {0: 'other/none', 1: 'individual', 2: 'organization/group', 3: 'community'}
SEVERITY_LABELS = {0: 'none', 1: 'low', 2: 'medium', 3: 'high'}

def predict(text, model=None, return_probs=False):
    """
    Predict hate type, target group, and severity for a given text.
    """
    if model is None:
        model = best_model_full
    
    model.eval()
    encoding = tokenizer(text, max_length=160, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
    
    ht_pred = ht_logits.argmax(dim=1).item()
    tg_pred = tg_logits.argmax(dim=1).item()
    sv_pred = sv_logits.argmax(dim=1).item()
    
    result = {
        'text': text[:100] + '...' if len(text) > 100 else text,
        'hate_type': HATE_TYPE_LABELS[ht_pred],
        'target_group': TARGET_GROUP_LABELS[tg_pred],
        'severity': SEVERITY_LABELS[sv_pred],
    }
    
    if return_probs:
        result['hate_type_probs'] = torch.softmax(ht_logits, dim=1).cpu().numpy()[0]
        result['target_group_probs'] = torch.softmax(tg_logits, dim=1).cpu().numpy()[0]
        result['severity_probs'] = torch.softmax(sv_logits, dim=1).cpu().numpy()[0]
    
    return result

# === TEST EXAMPLES ===
print('Testing predictions with CORRECTED labels...\n')

test_texts = [
    "তুই একটা বোকা",                    # Bengali - personal attack
    "You're such an idiot",            # English - personal attack  
    "tui ekta pagol",                  # Banglish - personal attack
    "Have a nice day!",                # English - not hate
    "এই দেশের মানুষ সব চোর",            # Bengali - could be political/community
]

for text in test_texts:
    result = predict(text)
    print(f"Text: {result['text']}")
    print(f"  → Hate Type: {result['hate_type']}, Target: {result['target_group']}, Severity: {result['severity']}")
    print()

Testing predictions with CORRECTED labels...

Text: তুই একটা বোকা
  → Hate Type: personal_attack, Target: individual, Severity: low

Text: তুই একটা বোকা
  → Hate Type: personal_attack, Target: individual, Severity: low

Text: You're such an idiot
  → Hate Type: personal_attack, Target: individual, Severity: low

Text: You're such an idiot
  → Hate Type: personal_attack, Target: individual, Severity: low

Text: tui ekta pagol
  → Hate Type: not_hate/other, Target: individual, Severity: none

Text: tui ekta pagol
  → Hate Type: not_hate/other, Target: individual, Severity: none

Text: Have a nice day!
  → Hate Type: not_hate/other, Target: individual, Severity: none

Text: Have a nice day!
  → Hate Type: not_hate/other, Target: individual, Severity: none

Text: এই দেশের মানুষ সব চোর
  → Hate Type: personal_attack, Target: individual, Severity: low

Text: এই দেশের মানুষ সব চোর
  → Hate Type: personal_attack, Target: individual, Severity: low



In [39]:
# 16. 🎯 Custom Prediction - Enter your own text!
# =====================================================
# Change the text below and run this cell to predict
# =====================================================

custom_text = "turja khanki"  # ← Change this to any text you want!

result = predict(custom_text, return_probs=True)

print(f"📝 Text: {result['text']}")
print(f"\n🎯 Predictions:")
print(f"   Hate Type: {result['hate_type']}")
print(f"   Target Group: {result['target_group']}")
print(f"   Severity: {result['severity']}")

print(f"\n📊 Confidence Scores:")
print(f"   Hate Type: {dict(zip(HATE_TYPE_LABELS.values(), [f'{p:.2%}' for p in result['hate_type_probs']]))}")
print(f"   Target Group: {dict(zip(TARGET_GROUP_LABELS.values(), [f'{p:.2%}' for p in result['target_group_probs']]))}")
print(f"   Severity: {dict(zip(SEVERITY_LABELS.values(), [f'{p:.2%}' for p in result['severity_probs']]))}")

📝 Text: turja khanki

🎯 Predictions:
   Hate Type: personal_attack
   Target Group: individual
   Severity: low

📊 Confidence Scores:
   Hate Type: {'not_hate/other': '6.50%', 'political': '0.92%', 'religious': '0.65%', 'gender': '0.06%', 'personal_attack': '90.72%', 'geopolitical': '1.16%'}
   Target Group: {'other/none': '1.31%', 'individual': '94.73%', 'organization/group': '3.11%', 'community': '0.85%'}
   Severity: {'none': '5.55%', 'low': '91.19%', 'medium': '2.90%', 'high': '0.36%'}


In [40]:
# 17. 🧪 Test Personal Attack + High Severity Examples
# =====================================================
# Expected: hate_type=personal_attack, target=individual, severity=high
# =====================================================

test_examples = [
    # Bengali
    ("তুই মরে যা শালা", "Bengali - death threat"),
    ("তোর বাপ-মা তোকে কেন জন্ম দিল", "Bengali - family insult"),
    ("তুই একটা জানোয়ার", "Bengali - dehumanizing"),
    
    # English
    ("I hope you die alone", "English - death wish"),
    ("You're a worthless piece of garbage", "English - dehumanizing"),
    ("Kill yourself you pathetic loser", "English - suicide incitement"),
    
    # Banglish
    ("tui ekta shala kutta", "Banglish - animal slur"),
    ("tor jonno duniya kharap", "Banglish - blame"),
    ("tui moira ja harami", "Banglish - death threat"),
    
    # Control - NOT hate
    ("আজকে আবহাওয়া খুব সুন্দর", "Bengali - neutral"),
    ("Have a nice day!", "English - neutral"),
]

print("🧪 Testing Personal Attack + High Severity Examples\n")
print("=" * 80)

for text, description in test_examples:
    result = predict(text)
    
    # Check if predictions match expected
    is_personal = "✅" if result['hate_type'] == 'personal_attack' else "❌"
    is_individual = "✅" if result['target_group'] == 'individual' else "⚠️"
    is_high = "✅" if result['severity'] == 'high' else "⚠️"
    
    print(f"\n📝 {description}")
    print(f"   Text: {text}")
    print(f"   Hate Type: {result['hate_type']} {is_personal}")
    print(f"   Target: {result['target_group']} {is_individual}")
    print(f"   Severity: {result['severity']} {is_high}")

print("\n" + "=" * 80)
print("Legend: ✅ = Expected | ⚠️ = Different | ❌ = Wrong category")

🧪 Testing Personal Attack + High Severity Examples


📝 Bengali - death threat
   Text: তুই মরে যা শালা
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 Bengali - death threat
   Text: তুই মরে যা শালা
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 Bengali - family insult
   Text: তোর বাপ-মা তোকে কেন জন্ম দিল
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 Bengali - family insult
   Text: তোর বাপ-মা তোকে কেন জন্ম দিল
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 Bengali - dehumanizing
   Text: তুই একটা জানোয়ার
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 Bengali - dehumanizing
   Text: তুই একটা জানোয়ার
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 English - death wish
   Text: I hope you die alone
   Hate Type: personal_attack ✅
   Target: individual ✅
   Severity: low ⚠️

📝 English - death wish


In [41]:
# 18. 🎯 Test Different Target Groups
# =====================================================
# Testing: individual, organization/group, community targets
# =====================================================

target_examples = [
    # === INDIVIDUAL (target_group=1) ===
    ("You're an idiot", "English - individual insult", "individual"),
    ("তুই একটা বোকা", "Bengali - individual insult", "individual"),
    ("That guy is a complete moron", "English - individual", "individual"),
    
    # === ORGANIZATION/GROUP (target_group=2) ===
    ("This company is full of thieves", "English - organization", "organization/group"),
    ("The government is corrupt and useless", "English - government/org", "organization/group"),
    ("এই দল সব চোর", "Bengali - political party", "organization/group"),
    ("Facebook is destroying society", "English - company", "organization/group"),
    
    # === COMMUNITY (target_group=3) ===
    ("All Muslims are terrorists", "English - religious community", "community"),
    ("হিন্দুরা সব খারাপ", "Bengali - religious community", "community"),
    ("Women belong in the kitchen", "English - gender community", "community"),
    ("Immigrants are ruining this country", "English - ethnic community", "community"),
    ("All politicians are liars", "English - occupational group", "community"),
    
    # === OTHER/NONE (target_group=0) - Neutral ===
    ("The weather is nice today", "English - neutral", "other/none"),
    ("আজকে আবহাওয়া ভালো", "Bengali - neutral", "other/none"),
]

print("🎯 Testing Different Target Groups\n")
print("=" * 80)

# Track accuracy per target
correct = {'individual': 0, 'organization/group': 0, 'community': 0, 'other/none': 0}
total = {'individual': 0, 'organization/group': 0, 'community': 0, 'other/none': 0}

for text, description, expected_target in target_examples:
    result = predict(text)
    
    total[expected_target] += 1
    is_correct = result['target_group'] == expected_target
    if is_correct:
        correct[expected_target] += 1
    
    icon = "✅" if is_correct else "❌"
    
    print(f"\n📝 {description}")
    print(f"   Text: {text}")
    print(f"   Expected Target: {expected_target}")
    print(f"   Predicted: {result['target_group']} {icon}")
    print(f"   (Hate Type: {result['hate_type']}, Severity: {result['severity']})")

print("\n" + "=" * 80)
print("\n📊 Accuracy by Target Group:")
for target in ['individual', 'organization/group', 'community', 'other/none']:
    acc = correct[target] / total[target] * 100 if total[target] > 0 else 0
    print(f"   {target}: {correct[target]}/{total[target]} ({acc:.0f}%)")

🎯 Testing Different Target Groups


📝 English - individual insult
   Text: You're an idiot
   Expected Target: individual
   Predicted: individual ✅
   (Hate Type: personal_attack, Severity: low)

📝 English - individual insult
   Text: You're an idiot
   Expected Target: individual
   Predicted: individual ✅
   (Hate Type: personal_attack, Severity: low)

📝 Bengali - individual insult
   Text: তুই একটা বোকা
   Expected Target: individual
   Predicted: individual ✅
   (Hate Type: personal_attack, Severity: low)

📝 Bengali - individual insult
   Text: তুই একটা বোকা
   Expected Target: individual
   Predicted: individual ✅
   (Hate Type: personal_attack, Severity: low)

📝 English - individual
   Text: That guy is a complete moron
   Expected Target: individual
   Predicted: individual ✅
   (Hate Type: personal_attack, Severity: low)

📝 English - individual
   Text: That guy is a complete moron
   Expected Target: individual
   Predicted: individual ✅
   (Hate Type: personal_attack, Severi

In [42]:
# 19. 📊 Analyze Training Data Distribution (Diagnose Model Issues)
# =====================================================

print("📊 TRAINING DATA CLASS DISTRIBUTION ANALYSIS\n")
print("=" * 60)

# Filter only valid labels (not -1)
train_with_ht = train_df[train_df['hate_type'] != -1]
train_with_tg = train_df[train_df['target_group'] != -1]
train_with_sv = train_df[train_df['severity'] != -1]

print(f"\n🏷️ HATE TYPE (n={len(train_with_ht)} samples with labels)")
print("-" * 40)
ht_counts = train_with_ht['hate_type'].value_counts().sort_index()
for idx, count in ht_counts.items():
    pct = count / len(train_with_ht) * 100
    label = HATE_TYPE_LABELS.get(idx, f'unknown_{idx}')
    bar = "█" * int(pct / 2)
    print(f"  {idx} ({label:15}): {count:5} ({pct:5.1f}%) {bar}")

print(f"\n🎯 TARGET GROUP (n={len(train_with_tg)} samples with labels)")
print("-" * 40)
tg_counts = train_with_tg['target_group'].value_counts().sort_index()
for idx, count in tg_counts.items():
    pct = count / len(train_with_tg) * 100
    label = TARGET_GROUP_LABELS.get(idx, f'unknown_{idx}')
    bar = "█" * int(pct / 2)
    print(f"  {idx} ({label:18}): {count:5} ({pct:5.1f}%) {bar}")

print(f"\n⚠️ SEVERITY (n={len(train_with_sv)} samples with labels)")
print("-" * 40)
sv_counts = train_with_sv['severity'].value_counts().sort_index()
for idx, count in sv_counts.items():
    pct = count / len(train_with_sv) * 100
    label = SEVERITY_LABELS.get(idx, f'unknown_{idx}')
    bar = "█" * int(pct / 2)
    print(f"  {idx} ({label:8}): {count:5} ({pct:5.1f}%) {bar}")

print("\n" + "=" * 60)
print("\n📋 DIAGNOSIS:")
print("  - If one class dominates (>50%), model will over-predict it")
print("  - Classes with <5% samples are hard to learn")
print("  - Missing labels (-1) reduce effective training data per task")

📊 TRAINING DATA CLASS DISTRIBUTION ANALYSIS


🏷️ HATE TYPE (n=11339 samples with labels)
----------------------------------------
  0 (not_hate/other ):  5331 ( 47.0%) ███████████████████████
  1 (political      ):   489 (  4.3%) ██
  2 (religious      ):   553 (  4.9%) ██
  4 (personal_attack):  3932 ( 34.7%) █████████████████
  5 (geopolitical   ):  1034 (  9.1%) ████

🎯 TARGET GROUP (n=5176 samples with labels)
----------------------------------------
  0 (other/none        ):   228 (  4.4%) ██
  1 (individual        ):  2978 ( 57.5%) ████████████████████████████
  2 (organization/group):  1306 ( 25.2%) ████████████
  3 (community         ):   664 ( 12.8%) ██████

⚠️ SEVERITY (n=45518 samples with labels)
----------------------------------------
  0 (none    ): 25592 ( 56.2%) ████████████████████████████
  1 (low     ): 19700 ( 43.3%) █████████████████████
  2 (medium  ):   226 (  0.5%) 


📋 DIAGNOSIS:
  - If one class dominates (>50%), model will over-predict it
  - Classes with <5